In [7]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
style.use("ggplot")
from textwrap import wrap
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from collections import Counter
import string
import nltk
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

from datasets import load_dataset


/Users/theodore/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/theodore/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# huggin face dataset
dataset1 = load_dataset("artem9k/ai-text-detection-pile")

df = pd.DataFrame(dataset1['train'].to_pandas())
df = df[['text','source']]
df['source'] = df['source'].map({'human': 0, 'ai': 1})

In [25]:
# using m1 mac
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [29]:
print(df)



                                                      text  source
0        12 Years a Slave: An Analysis of the Film Essa...       0
1        20+ Social Media Post Ideas to Radically Simpl...       0
2        2022 Russian Invasion of Ukraine in Global Med...       0
3        533 U.S. 27 (2001) Kyllo v. United States: The...       0
4        A Charles Schwab Corporation Case Essay\n\nCha...       0
...                                                    ...     ...
1392517  Today, I accomplished a major feat. I stepped ...       1
1392518  As rockets rain down from the sky\nEurope trem...       1
1392519  On January 6th, 2023, the world lost a true pi...       1
1392520  A gene bank is a repository of genetic materia...       1
1392521  On the twelfth day of Christmas, My true love ...       1

[1392522 rows x 2 columns]


In [30]:
# 60% Training, 20% Validation, 20% Test
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)  
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  

print("Training set size:", len(train_df))
print("Validation set size:", len(valid_df))
print("Test set size:", len(test_df))

Training set size: 835513
Validation set size: 278504
Test set size: 278505


In [32]:
df0_text = train_df[train_df['source'] == 0].copy()
df1_ai = train_df[train_df['source'] == 1].copy()

train_df = pd.concat([df0_text,df1_ai], ignore_index=True)
train_df

/var/folders/7_/v74xm1tn6zn5xx52z3f4_9b40000gn/T/ipykernel_4947/1483335269.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df0_text = train_df[df['source'] == 0].copy()
/var/folders/7_/v74xm1tn6zn5xx52z3f4_9b40000gn/T/ipykernel_4947/1483335269.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1_ai = train_df[df['source'] == 1].copy()


In [ ]:
nltk.download('vader_lexicon')

In [ ]:
# establish stop words and linking words
stop_words = set(stopwords.words('english'))
linking_words = {'to', 'the', 'and', 'of', 'in', 'on', 'for', 'with', 'at', 'a', 'an'}

# sentiment analyzer
sid = SentimentIntensityAnalyzer()

def analyze_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token not in linking_words]
    tokens = [token for token in tokens if token.isalpha()]
    sentiment_scores = sid.polarity_scores(text)
    return len(tokens), sentiment_scores['compound']

df0_text[['text_length', 'sentiment_score']] = df0_text['text'].apply(lambda x: pd.Series(analyze_text(x)))
df1_ai[['text_length', 'sentiment_score']] = df0_text['text'].apply(lambda x: pd.Series(analyze_text(x)))